# Pipeline for AutoML training

Azure Machine Learning Pipeline を利用して、再利用可能なパイプラインを作成することができます。本 Notebook では AutoML のモデル構築のパイプラインを作成します。

## 1. 事前準備

## 1.1 Python SDK のインポート

Azure Machine Learning の Python SDK などをインポートします。

In [ ]:
import logging
import os
import shutil

import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.run import Run
from azureml.widgets import RunDetails
from azureml.core.model import Model 
from azureml.train.automl import AutoMLConfig

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig

Azure ML Python SDK のバージョンが 1.8.0 以上になっていることを確認します。

In [ ]:
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

## 1.2 Azure ML Workspace との接続

In [ ]:
ws = Workspace.from_config()

In [ ]:
# 実験名の指定
experiment_name = 'livedoor-news-classification-BERT-pipeline-train'

experiment = Experiment(ws, experiment_name)

output = {}
#output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

## 1.3 計算環境の準備

BERT を利用するための GPU の `Compute Cluster` を準備します。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute Cluster の名称
amlcompute_cluster_name = "gpucluster"

# クラスターの存在確認
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    
except ComputeTargetException:
    print('指定された名称のクラスターが見つからないので新規に作成します.')
    compute_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_NC6_V3",
                                                           max_nodes = 4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## 1.4 学習データの準備
今回は **livedoor ニュースのコーパス** を学習データとして利用します。ニュースのカテゴリー分類のモデルを構築します。  

In [ ]:
target_column_name = 'label'  # カテゴリーの列 
feature_column_name = 'text'  # ニュース記事の列

In [ ]:
train_dataset = Dataset.get_by_name(ws, "livedoor").keep_columns(["text","label"])
train_dataset.take(5).to_pandas_dataframe()

# 2. 機械学習パイプライン
## 2.1 AutoML の設定

自動機械学習 Automated Machine Learning の設定と学習を行っていきます。

In [ ]:
from azureml.automl.core.featurization import FeaturizationConfig
featurization_config = FeaturizationConfig()
# テキストデータの言語を指定します。日本語の場合は "jpn" と指定します。
featurization_config = FeaturizationConfig(dataset_language="jpn") # 英語の場合は下記をコメントアウトしてください。

In [ ]:
# 明示的に `text` の列がテキストデータであると指定します。
featurization_config.add_column_purpose('text', 'Text')
featurization_config.blocked_transformers = ['TfIdf','CountVectorizer']  # BERT のみを利用したい場合はコメントアウトを外します

In [ ]:
project_folder = './project'

In [ ]:
# 自動機械学習の設定
automl_settings = {
    "experiment_timeout_hours" : 0.5,  # 学習時間 (hour)
    "primary_metric": 'accuracy',  # 評価指標
    "max_concurrent_iterations": 4,  # 計算環境の最大並列数 
    "max_cores_per_iteration": -1,
    "enable_dnn": True, # 深層学習を有効
    "enable_early_stopping": False,
    "validation_size": 0.2,
    "verbosity": logging.INFO,
    "force_text_dnn": True,
    #"n_cross_validations": 5,
}

automl_config = AutoMLConfig(task = 'classification', 
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             featurization=featurization_config,
                             path = project_folder,  # 必須パラメータ
                             **automl_settings
                            )

## 2.2 パイプラインの作成

In [ ]:
from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import PipelineData, TrainingOutput

In [ ]:
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

In [ ]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [ ]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

## 2.3 パイプラインの検証

In [ ]:
pipeline.validate()
print("Pipeline validation complete")

## 2.4 パイプラインの実行

In [ ]:
pipeline_run = experiment.submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()